Issues:

1) test_image_14 : two squares treated as one, due to which undefined centroid key value in color dict, also the only **orange square** being detected is giving an answer- **Red** *(refer issue and issue1 images)*

2) test_image_15 : **one orange triangle** detected as **blue** triangle *(refer issue2 image)*

Maybe the issue lies in orange contouring, but I tried every permutation to get it through

In [13]:
'''
*****************************************************************************************
*
*        		===============================================
*           		Berryminator (BM) Theme (eYRC 2021-22)
*        		===============================================
*
*  This script is to implement Task 1A of Berryminator(BM) Theme (eYRC 2021-22).
*  
*  This software is made available on an "AS IS WHERE IS BASIS".
*  Licensee/end user indemnifies and will keep e-Yantra indemnified from
*  any and all claim(s) that emanate from the use of the Software or 
*  breach of the terms of this agreement.
*
*****************************************************************************************
'''

# Team ID:			BM_1424
# Author List:		Uzma Khan, Shairin Meraj, Abbas Haider, Faizan Choudhary
# Filename:			task_1a.py
# Functions:		detect_shapes, get_labeled_image, 
# 					[ Comma separated list of functions in this file ]


####################### IMPORT MODULES #######################
## You are not allowed to make any changes in this section. ##
## You have to implement this task with the three available ##
## modules for this task (numpy, opencv, os)                ##
##############################################################
import cv2
import numpy as np
import os
##############################################################

################# ADD UTILITY FUNCTIONS HERE #################

def centroid(contour):
    """
    Purpose:
    ---
    This function takes contour as an argument and returns a list containing
    the x and y values of the centroid

    Input Arguments:
    ---
    `contour` : [ numpy array ]
            numpy array of (x, y) coordinates of boundary points of the object.

    Returns:
    ---
    `centroid` : [ list ]
            list of (x,y) coordinates of the centroid of contours
    """
    
    centroid = []

    M = cv2.moments(contour)
    if M['m00'] != 0.0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])

        centroid.append((cx,cy))

    return centroid


def colors_detected(img):
    """
    Purpose:
    ---
    This function takes the image as argument and returns a dictionary
    denoting the color of the shapes in the image.

    Input Arguments:
    ---
    `img` : [ numpy array ]
            numpy array of image returned by cv2 library

    Returns:
    ---
    `detected_colors` : {dictionary}
            dictionary containing details of colors present in image
    """
    
    detected_colors = {}
                
    # Convert BGR to HSV
    hsv_frame = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # defining color ranges

    # red color range
    low_red = np.array([0, 150, 50], np.uint8)
    high_red = np.array([10, 255, 255], np.uint8)
    red_mask = cv2.inRange(hsv_frame, low_red, high_red)

    # blue color range
    low_blue = np.array([94, 80, 2], np.uint8)
    high_blue = np.array([126, 255, 255], np.uint8)
    blue_mask = cv2.inRange(hsv_frame, low_blue, high_blue)

    # green color range
    low_green = np.array([36, 25, 25], np.uint8)
    high_green = np.array([70, 255, 255], np.uint8)
    green_mask = cv2.inRange(hsv_frame, low_green, high_green)

    # orange color range
    low_orange = np.array([15, 150, 50], np.uint8)
    high_orange = np.array([25, 255, 255], np.uint8)
    orange_mask = cv2.inRange(hsv_frame, low_orange, high_orange)

    # Morphological Transform: Dilation, to remove noises from the images and performing bitwise_and function to specifically detect a particular color.

    kernal = np.ones((6, 6), "uint8")
      
    # For red color
    red_mask = cv2.dilate(red_mask, kernal)
      
    # For green color
    green_mask = cv2.dilate(green_mask, kernal)
      
    # For blue color
    blue_mask = cv2.dilate(blue_mask, kernal)

    # For orange color
    orange_mask = cv2.dilate(orange_mask, kernal)
    
    # Contours for color detection

    # Creating contour to track red color
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Creating contour to track green color
    green_contours, _ = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  
    # Creating contour to track blue color
    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    # Creating contour to track orange color
    orange_contours, _ = cv2.findContours(orange_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # RED
    for contour in red_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]

            detected_colors[cx,cy] = ['Red']
    
    #GREEN
    for contour in green_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]


            detected_colors[cx,cy] = ['Green']
        
    #BLUE
    for contour in blue_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]


            detected_colors[cx,cy] = ['Blue']
        
    #ORANGE
    for contour in orange_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]


            detected_colors[cx,cy] = ['Orange']

    return detected_colors


def shapes(img):
    """
    Purpose:
    ---
    This function takes the image as argument and returns a dictionary
    denoting the shapes in the image.

    Input Arguments:
    ---
    `img` : [ numpy array ]
            numpy array of image returned by cv2 library

    Returns:
    ---
    `img_shapes` : {dictionary}
            dictionary containing details of shapes present in image
    """
    
    img_shapes = {}
    
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Thresholding
    _, threshold = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)

    # Contouring
    contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #to loop over the contours
    i=0
    for cnt in contours:
    
        # ignoring first counter because 
        # findcontour function detects whole image as one shape
        if i==0:
            i=1
            continue

        # approximating
        approx = cv2.approxPolyDP(cnt, 0.025*cv2.arcLength(cnt, True), True)

        # centroid 
        centroids = centroid(cnt)
   
        cx = centroids[0][0]
        cy = centroids[0][1]
        
        # putting shape names corresponding to centroid key
        if len(approx) == 3:
            img_shapes[cx,cy] = ['Triangle']
  
        elif len(approx) == 4:
        
            # for distinguishing between rectangle and square
            x, y, w, h = cv2.boundingRect(approx)

            aspectRatio = float (w)/h
            if aspectRatio >= 0.95 and aspectRatio <= 1.05:
                img_shapes[cx,cy] = ['Square']
            else:
                img_shapes[cx,cy] = ['Rectangle']

        elif (len(approx) == 5):
             img_shapes[cx,cy] = ['Pentagon']
  
        else:
             img_shapes[cx,cy] = ['Circle']

    return img_shapes


##############################################################

def detect_shapes(img):

	"""
	Purpose:
	---
	This function takes the image as an argument and returns a nested list
	containing details of colored (non-white) shapes in that image

	Input Arguments:
	---
	`img` :	[ numpy array ]
			numpy array of image returned by cv2 library

	Returns:
	---
	`detected_shapes` : [ list ]
			nested list containing details of colored (non-white) 
			shapes present in image
	
	Example call:
	---
	shapes = detect_shapes(img)
	"""    
	detected_shapes = []

	##############	ADD YOUR CODE HERE	##############
	
	color = colors_detected(img)

	shape = shapes(img)
    
	keys_color = list(color.keys())
    
	keys_shape = list(shape.keys())
    

	for key_shp in keys_shape:

		details_list = []
		cx_key = key_shp[0]
		cy_key = key_shp[1]
        
    # error inclusion of 1% to account for key matching with color and shape dicts
		upper_cx = 1.01 * cx_key  
		lower_cx = 0.99 * cx_key
    
		upper_cy = 1.01 * cy_key  
		lower_cy = 0.99 * cy_key

		for key_clr in keys_color:
			cx_k = key_clr[0]
			cy_k = key_clr[1]
            
			if (lower_cx <= cx_k <= upper_cx or lower_cy <= cy_k <= upper_cy):
				temp1 = (cx_k, cy_k)
				temp2 = (cx_key, cy_key)
				details_list.append(color.get(temp1)[0])
				details_list.append(shape.get(temp2)[0])
				details_list.append(temp2)
				detected_shapes.append(details_list)
                # upper bound to prevent list items to go beyond 3 elements
				if len(details_list) == 3 :
					break
    

	##################################################
	
	return detected_shapes

def get_labeled_image(img, detected_shapes):
	######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
	"""
	Purpose:
	---
	This function takes the image and the detected shapes list as an argument
	and returns a labelled image

	Input Arguments:
	---
	`img` :	[ numpy array ]
			numpy array of image returned by cv2 library

	`detected_shapes` : [ list ]
			nested list containing details of colored (non-white) 
			shapes present in image

	Returns:
	---
	`img` :	[ numpy array ]
			labelled image
	
	Example call:
	---
	img = get_labeled_image(img, detected_shapes)
	"""
	######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########    

	for detected in detected_shapes:
		colour = detected[0]
		shape = detected[1]
		coordinates = detected[2]
		cv2.putText(img, str((colour, shape)),coordinates, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
	return img

if __name__ == '__main__':
	
	# path directory of images in 'test_images' folder
	img_dir_path = 'test_images/'

	# path to 'test_image_1.png' image file
	file_num = 1
	img_file_path = img_dir_path + 'test_image_' + str(file_num) + '.png'
	
	# read image using opencv
	img = cv2.imread(img_file_path)
	
	print('\n============================================')
	print('\nFor test_image_' + str(file_num) + '.png')
	
	# detect shape properties from image
	detected_shapes = detect_shapes(img)
	print(detected_shapes)
	
	# display image with labeled shapes
	img = get_labeled_image(img, detected_shapes)
	cv2.imshow("labeled_image", img)
	cv2.waitKey(2000)
	cv2.destroyAllWindows()
	
	choice = input('\nDo you want to run your script on all test images ? => "y" or "n": ')
	
	if choice == 'y':

		for file_num in range(1, 16):
			
			# path to test image file
			img_file_path = img_dir_path + 'test_image_' + str(file_num) + '.png'
			
			# read image using opencv
			img = cv2.imread(img_file_path)
	
			print('\n============================================')
			print('\nFor test_image_' + str(file_num) + '.png')
			
			# detect shape properties from image
			detected_shapes = detect_shapes(img)
			print(detected_shapes)
			
			# display image with labeled shapes
			img = get_labeled_image(img, detected_shapes)
			cv2.imshow("labeled_image", img)
			cv2.waitKey(2000)
			cv2.destroyAllWindows()






For test_image_1.png
[['Red', 'Circle', (588, 370)], ['Green', 'Square', (325, 145)]]

Do you want to run your script on all test images ? => "y" or "n": y


For test_image_1.png
[['Red', 'Circle', (588, 370)], ['Green', 'Square', (325, 145)]]


For test_image_2.png
[['Blue', 'Circle', (274, 348)], ['Blue', 'Pentagon', (648, 420)], ['Green', 'Triangle', (766, 165)], ['Red', 'Rectangle', (301, 118)]]


For test_image_3.png
[['Green', 'Rectangle', (467, 467)], ['Blue', 'Square', (425, 272)], ['Green', 'Circle', (290, 110)], ['Red', 'Triangle', (643, 194)]]


For test_image_4.png
[['Blue', 'Square', (325, 451)], ['Red', 'Rectangle', (249, 199)], ['Blue', 'Circle', (588, 226)]]


For test_image_5.png
[['Red', 'Rectangle', (648, 398)], ['Green', 'Circle', (308, 370)], ['Blue', 'Circle', (631, 149)], ['Blue', 'Square', (183, 105)]]


For test_image_6.png
[['Blue', 'Circle', (274, 348)], ['Blue', 'Pentagon', (648, 420)], ['Green', 'Triangle', (766, 165)], ['Red', 'Rectangle', (301, 118)]]

